In [5]:
# 引入模組
import numpy as np
import pandas as pd
import datetime
import pymysql

def dbConnection():
    # 建立MySQL連線
    dbBase = pymysql.connect(host='XXXXX', port=3306, user='root', passwd='XXXXXXX', db='ResourceApp', charset='utf8')
    return dbBase

# 框範圍
def dataResource(dbBase, startDate, endDate):
    sql = "SELECT R.ip, R.ReceiveTime, R.hrSWRunName, concat(hrSWRunPath, hrSWRunName) as Paths, H.SoftName, H.SoftGroup1,H.SoftGroup2, C.SoftClass asSoftClassWare, C.SoftType, C.Unit, C.SoftApp, R.hrSWRunPerfCPU, R.hrSWRunPerfMem, \
    CASE \
    WHEN H.SoftGroup1 is not null THEN 1 \
    WHEN SP.SoftName is not null THEN 2 \
    ELSE 0 \
    END AS SystemType, \
    now() \
    FROM ResourceApp.HrSWRun AS R \
    LEFT join ResourceApp.ConfSoftHistoryList AS H \
    ON R.hrSWRunName = H.SoftRun AND concat(R.hrSWRunPath, R.hrSWRunName) = H.SoftPath \
    LEFT JOIN ResourceApp.ConfSoftwareList AS C \
    ON C.SoftName = H.SoftName \
    LEFT JOIN ResourceApp.SystemProcess as SP \
    ON R.hrSWRunName = SP.SoftRun \
    where R.ReceiveTime >= str_to_date('{0}', '%Y-%m-%d %H:%i:%s') AND R.ReceiveTime < str_to_date('{1}', '%Y-%m-%d %H:%i:%s')".format(startDate, endDate)
    # print(sql)
    cursor = dbBase.cursor()
    cursor.execute(sql)
    dataResult = cursor.fetchall()
    return dataResult

def SoftProcessInfoSQL():
    sql = "INSERT INTO ResourceApp.SoftProcessInfo (ip,ReceiveTime,hrSWRunName,ProcessPath,SoftName,SoftGroup1,SoftGroup2,SoftClass,SoftType,Unit,SoftApp,hrSWRunPerfCPU,hrSWRunPerfMem,SystemType,CreateTime) \
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    return sql

if __name__ == "__main__":
    # 建立連線
    dbBase = dbConnection()
    # 撈範圍
    dataResult = dataResource(dbBase, "2020-07-25", "2020-07-26")
    print(type(dataResult))
    cursor = dbBase.cursor()
    try:
        sqlCMD = SoftProcessInfoSQL()
        cursor.executemany(sqlCMD, dataResult)
        dbBase.commit()
    except pymysql.Error as e:
        print("Err:", e)
    finally:
        #關閉連線
        cursor.close()
        dbBase.close()

    print("Done!!")



<class 'tuple'>
Done!!
